In [151]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split

In [152]:
# util functions
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


def pattern_accuracy(y_true, y_pred):
    return np.sum(np.all(y_true == y_pred, axis=1)) / y_true.shape[0]


tf.keras.metrics.BinaryAccuracy(
    name="binary_accuracy", dtype=None, threshold=0.5
)
binary_accuracy = tf.keras.metrics.BinaryAccuracy()

In [153]:
# import data
raw_data = loadarff("emotions_train.arff")
df = pd.DataFrame(raw_data[0])

In [154]:
# preprocess data, normalize input to 0-1 scale
X = df.iloc[:,:-6]
norm_X = normalize(X)

y = df.iloc[:, 72:]
for col in y.columns:
    y[col] = y[col].apply(lambda x: int(x.decode('utf-8')))

In [155]:
# split data into train/val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [156]:
# build model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='sigmoid'))
model.add(Dense(y_train.shape[1], kernel_initializer='uniform', activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 512)               37376     
                                                                 
 dense_35 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 40,454
Trainable params: 40,454
Non-trainable params: 0
_________________________________________________________________


In [157]:
# train
history = model.fit(X_train, y_train,
                    batch_size=8,
                    epochs=100,
                    verbose=1,
                    validation_split=.2,
                    callbacks=[callback])

Epoch 1/100
35/35 [==============================] - 1s 7ms/step - loss: 0.6402 - binary_accuracy: 0.6679 - val_loss: 0.6282 - val_binary_accuracy: 0.7254
Epoch 2/100
35/35 [==============================] - 0s 3ms/step - loss: 0.6123 - binary_accuracy: 0.6917 - val_loss: 0.5520 - val_binary_accuracy: 0.7254
Epoch 3/100
35/35 [==============================] - 0s 3ms/step - loss: 0.6057 - binary_accuracy: 0.6994 - val_loss: 0.5616 - val_binary_accuracy: 0.7254
Epoch 4/100
35/35 [==============================] - 0s 3ms/step - loss: 0.5818 - binary_accuracy: 0.7030 - val_loss: 0.5332 - val_binary_accuracy: 0.7371
Epoch 5/100
35/35 [==============================] - 0s 3ms/step - loss: 0.5733 - binary_accuracy: 0.6988 - val_loss: 0.5019 - val_binary_accuracy: 0.7770
Epoch 6/100
35/35 [==============================] - 0s 3ms/step - loss: 0.5504 - binary_accuracy: 0.7250 - val_loss: 0.5047 - val_binary_accuracy: 0.7254
Epoch 7/100
35/35 [==============================] - 0s 3ms/step - los

In [158]:
# save or load model
model.save('output/emotions-model.h5')
model = load_model('output/emotions-model.h5')

In [159]:
# predict
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

In [160]:
# evaluate
print("Binary accuracy: ", binary_accuracy(y_test, y_pred))
print("Pattern accuracy: ",pattern_accuracy(y_test, y_pred))

Binary accuracy:  tf.Tensor(0.7916667, shape=(), dtype=float32)
Pattern accuracy:  0.25
